In [1]:
import os,sys
from sklearn import preprocessing

import json
import codecs
import yaml
import numpy as np
import pandas as pd

sys.path.append('../utils')
from cudaMetrics import *

In [2]:
#
# read metrics folder
#
metrics_folder='home-gpu0'
appTrace = os.listdir(metrics_folder)
print len(appTrace)

76


In [3]:
appTrace

['metrics_cudasdk_interval.csv',
 'metrics_rodinia_hotspot.csv',
 'metrics_rodinia_heartwall.csv',
 'metrics_cudasdk_dxtc.csv',
 'metrics_cudasdk_MCSingleAsianOptionP.csv',
 'metrics_parboil_stencil.csv',
 'metrics_cudasdk_mergeSort.csv',
 'metrics_rodinia_hybridsort.csv',
 'metrics_shoc_lev1BFS.csv',
 'metrics_parboil_mriq.csv',
 'metrics_rodinia_b+tree.csv',
 'metrics_poly_covariance.csv',
 'metrics_cudasdk_shflscan.csv',
 'metrics_cudasdk_transpose.csv',
 'metrics_poly_gemm.csv',
 'metrics_cudasdk_dwtHaar1D.csv',
 'metrics_cudasdk_MCEstimatePiQ.csv',
 'metrics_poly_2dconv.csv',
 'metrics_cudasdk_convolutionFFT2D.csv',
 'metrics_rodinia_dwt2d.csv',
 'metrics_parboil_lbm.csv',
 'metrics_cudasdk_convolutionSeparable.csv',
 'metrics_cudasdk_concurrentKernels.csv',
 'metrics_poly_fdtd2d.csv',
 'metrics_cudasdk_sortingNetworks.csv',
 'metrics_poly_gesummv.csv',
 'metrics_rodinia_lud.csv',
 'metrics_poly_correlation.csv',
 'metrics_parboil_bfs.csv',
 'metrics_cudasdk_MCEstimatePiInlineP.cs

In [4]:
app_kernel_metrics_dd = {}
for i,f in enumerate(appTrace):
    csvfile  = metrics_folder + '/' + f
    appName = f[8:-4]
    
    #sep_str = f.split('_')
    #appName = sep_str[-1][:-4]
    #print sep_str
 
    
    df_app = read_trace(csvfile)
    
    kernelList = df_app['Kernel'].unique()
    
    #
    # save metrics set
    #
    if i == 0:
        metricsAll = {}
        metricsAll['current'] = list(df_app['Metric Name'].unique())
        with codecs.open('current_metrics.json', 'w', encoding='utf-8') as outfile:
            json.dump(metricsAll, outfile, ensure_ascii=False)
    
    # extract metrics for each kernel
    kernel_metrics_dd = {}
    for kernName in kernelList:
        df_kern = df_app.loc[df_app['Kernel'] == kernName]
        # go through each metrics and read the avg value
        metrics_dd = {}
        for index, rows in df_kern.iterrows():
            local_metric_name = rows['Metric Name']
            local_metric_value = rows['Avg']
            try:
                adjustedV = adjust_metric(local_metric_name, local_metric_value)
            except:
                print("Unexpected error!\nfile: {}\nmetric: {}\nvalue: {}".format(currentFile, 
                    local_metric_name, local_metric_value))
                sys.exit(0)
            
            metrics_dd[local_metric_name] = adjustedV
        # update 
        kernel_metrics_dd[kernName] = metrics_dd
    # update
    app_kernel_metrics_dd[appName] = kernel_metrics_dd
    #break

In [5]:
print("applications : {}".format(len(app_kernel_metrics_dd)))

totalKernNum = 0
for app, kern_metrics_dd in app_kernel_metrics_dd.iteritems():
    #print len(kern_metrics_dd)
    totalKernNum += len(kern_metrics_dd)

print("kernels : {}".format(str(totalKernNum)))

applications : 76
kernels : 238


### example

In [6]:
app_kernel_metrics_dd['cudasdk_scan']

{'scanExclusiveShared(uint4*, uint4*, unsigned int)': {'achieved_occupancy': 0.945988,
  'atomic_transactions': 0.0,
  'atomic_transactions_per_request': 0.0,
  'branch_efficiency': 1.0,
  'cf_executed': 419719.0,
  'cf_fu_utilization': 0.1,
  'cf_issued': 419719.0,
  'double_precision_fu_utilization': 0.0,
  'dram_read_throughput': 43.566339,
  'dram_read_transactions': 851990.0,
  'dram_utilization': 0.9,
  'dram_write_throughput': 43.50394,
  'dram_write_transactions': 850785.0,
  'ecc_throughput': 0.0,
  'ecc_transactions': 0.0,
  'eligible_warps_per_cycle': 1.124316,
  'flop_count_dp': 0.0,
  'flop_count_dp_add': 0.0,
  'flop_count_dp_fma': 0.0,
  'flop_count_dp_mul': 0.0,
  'flop_count_sp': 0.0,
  'flop_count_sp_add': 0.0,
  'flop_count_sp_fma': 0.0,
  'flop_count_sp_mul': 0.0,
  'flop_count_sp_special': 0.0,
  'flop_dp_efficiency': 0.0,
  'flop_sp_efficiency': 0.0,
  'gld_efficiency': 1.0,
  'gld_requested_throughput': 43.564476,
  'gld_throughput': 43.564476,
  'gld_transaction

### offline training

In [7]:
#
# read metrics
#
with open('current_metrics.json', 'r') as metricsFile:
     metricsAll = yaml.safe_load(metricsFile)
        
TargetMetrics = metricsAll['current']

In [8]:
featMatCols = ['KernelName']         
featMatCols.extend(TargetMetrics)

In [9]:
featMatCols

['KernelName',
 'sm_efficiency',
 'achieved_occupancy',
 'ipc',
 'issued_ipc',
 'inst_per_warp',
 'branch_efficiency',
 'warp_execution_efficiency',
 'warp_nonpred_execution_efficiency',
 'inst_replay_overhead',
 'issue_slot_utilization',
 'shared_load_transactions_per_request',
 'shared_store_transactions_per_request',
 'local_load_transactions_per_request',
 'local_store_transactions_per_request',
 'gld_transactions_per_request',
 'gst_transactions_per_request',
 'shared_store_transactions',
 'shared_load_transactions',
 'local_load_transactions',
 'local_store_transactions',
 'gld_transactions',
 'gst_transactions',
 'dram_read_transactions',
 'dram_write_transactions',
 'global_hit_rate',
 'local_hit_rate',
 'gld_requested_throughput',
 'gst_requested_throughput',
 'gld_throughput',
 'gst_throughput',
 'dram_read_throughput',
 'dram_write_throughput',
 'tex_cache_throughput',
 'local_load_throughput',
 'local_store_throughput',
 'shared_load_throughput',
 'shared_store_throughput',

In [10]:
#
#  totalKernNum  x featureDim
#
df_featureMat = pd.DataFrame(index=np.arange(0, totalKernNum), columns=featMatCols)
print df_featureMat.shape

(238, 114)


In [11]:
#
# export data to data frame, so that we can export to csv file easily
#
rowId = 0
for appName, kern_metrics_dd in app_kernel_metrics_dd.iteritems():
    for kernName, metrics_dd in kern_metrics_dd.iteritems():
        app_kern_name = '[[ '+ str(appName) + ' ]] ' + str(kernName)
        
        # add kernel name to the list
        df_featureMat.loc[rowId, 'KernelName'] = app_kern_name

        # add other metrics according to the column order
        for eachMetric in TargetMetrics:
            try:                
                df_featureMat.loc[rowId, eachMetric] = metrics_dd[eachMetric]
            except Exception as e:
                print e.message, e.args
                print('ERROR!! App = {}, Kernel={}'.format(appName, kernName))
                sys.exit(0)

        rowId += 1

In [12]:
df_featureMat

,KernelName,sm_efficiency,achieved_occupancy,ipc,issued_ipc,inst_per_warp,branch_efficiency,warp_execution_efficiency,warp_nonpred_execution_efficiency,inst_replay_overhead,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,[[ rodinia_b+tree ]] findRangeK,0.998196,0.946643,1.58373,1.5873,235.138,1,0.976755,0.864836,3.5e-05,...,0.3,0,0,0.1,0,0,0.1,2.59372,0,0
1,[[ rodinia_b+tree ]] findK,0.99834,0.853255,1.36619,1.33891,130.584,1,0.945485,0.928101,3.7e-05,...,0.2,0,0,0.1,0,0,0.1,2.1729,0,0
2,[[ cudasdk_threadFenceReduction ]] void reduce...,0.922423,0.657533,0.605605,0.607236,945.559,1,0.997663,0.987826,0.002115,...,0.3,0.035286,128,0.1,0,0,0.1,0.669109,0.00957817,0
3,[[ cudasdk_threadFenceReduction ]] void reduce...,0.930337,0.654782,0.647716,0.654897,929,1,0.998697,0.994113,0.002153,...,0.3,0,0,0.1,0,0,0.1,0.71753,0.0105205,0
4,[[ cudasdk_convolutionFFT2D ]] spPostprocess2D...,0.99213,0.831236,0.911174,0.910572,89.1887,1,0.9864,0.963975,0.000263,...,0.3,0,0,0.1,0,0,0.1,1.38569,0.026586,0
5,[[ cudasdk_convolutionFFT2D ]] __nv_static_73_...,0.992211,0.787109,1.62028,1.6197,158.312,1,0.985849,0.970033,0.000218,...,0.3,0,0,0.1,0,0,0.1,3.3234,0.05388,0
6,[[ cudasdk_convolutionFFT2D ]] void spVector10...,0.98964,0.359614,0.948036,0.945238,739,1,1,0.998647,0.00025,...,0.4,0,0,0.1,0,0,0.1,1.44303,0.0915731,0
7,[[ cudasdk_convolutionFFT2D ]] void spVector10...,0.990028,0.359621,0.941606,0.939806,739,1,1,0.998647,0.00022,...,0.4,0,0,0.1,0,0,0.1,1.42333,0.0915275,0
8,[[ cudasdk_convolutionFFT2D ]] void spRadix003...,0.990963,0.584691,1.17638,1.17876,464,1,1,0.991379,0.000271,...,0.3,0,0,0.1,0,0,0.1,2.82362,0.0831347,0
9,[[ cudasdk_convolutionFFT2D ]] modulateAndNorm...,0.994889,0.919004,0.340499,0.340512,24,1,1,0.958333,0.000484,...,0.3,0,0,0.1,0,0,0.1,0.367882,0.0141289,0


In [13]:
#
# drop the 1st column 
#

df_featureMat = df_featureMat.convert_objects(convert_numeric=True)

df_featureMat_cp = df_featureMat.copy()
df_featureMat_cp = df_featureMat_cp.drop('KernelName', axis=1)   # remove the 1st column

/home/leiming/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """


In [14]:
df_featureMat_cp

,sm_efficiency,achieved_occupancy,ipc,issued_ipc,inst_per_warp,branch_efficiency,warp_execution_efficiency,warp_nonpred_execution_efficiency,inst_replay_overhead,issue_slot_utilization,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,0.998196,0.946643,1.583730,1.587302,235.137979,1.000000,0.976755,0.864836,0.000035,0.357800,...,0.3,0.000000,0.0,0.1,0.0,0.0,0.1,2.593716,0.000000,0.000000
1,0.998340,0.853255,1.366189,1.338913,130.583663,1.000000,0.945485,0.928101,0.000037,0.303649,...,0.2,0.000000,0.0,0.1,0.0,0.0,0.1,2.172895,0.000000,0.000000
2,0.922423,0.657533,0.605605,0.607236,945.558594,1.000000,0.997663,0.987826,0.002115,0.130326,...,0.3,0.035286,128.0,0.1,0.0,0.0,0.1,0.669109,0.009578,0.000000
3,0.930337,0.654782,0.647716,0.654897,929.000000,1.000000,0.998697,0.994113,0.002153,0.140291,...,0.3,0.000000,0.0,0.1,0.0,0.0,0.1,0.717530,0.010521,0.000000
4,0.992130,0.831236,0.911174,0.910572,89.188660,1.000000,0.986400,0.963975,0.000263,0.214725,...,0.3,0.000000,0.0,0.1,0.0,0.0,0.1,1.385690,0.026586,0.000000
5,0.992211,0.787109,1.620280,1.619695,158.312500,1.000000,0.985849,0.970033,0.000218,0.394375,...,0.3,0.000000,0.0,0.1,0.0,0.0,0.1,3.323400,0.053880,0.000000
6,0.989640,0.359614,0.948036,0.945238,739.000000,1.000000,1.000000,0.998647,0.000250,0.207538,...,0.4,0.000000,0.0,0.1,0.0,0.0,0.1,1.443029,0.091573,0.000000
7,0.990028,0.359621,0.941606,0.939806,739.000000,1.000000,1.000000,0.998647,0.000220,0.206344,...,0.4,0.000000,0.0,0.1,0.0,0.0,0.1,1.423329,0.091528,0.000000
8,0.990963,0.584691,1.176382,1.178758,464.000000,1.000000,1.000000,0.991379,0.000271,0.273736,...,0.3,0.000000,0.0,0.1,0.0,0.0,0.1,2.823621,0.083135,0.000000
9,0.994889,0.919004,0.340499,0.340512,24.000000,1.000000,1.000000,0.958333,0.000484,0.081583,...,0.3,0.000000,0.0,0.1,0.0,0.0,0.1,0.367882,0.014129,0.000000


In [15]:
scaler = preprocessing.MinMaxScaler()
scaled_df = scaler.fit_transform(df_featureMat_cp)

In [16]:
scaled_df = pd.DataFrame(scaled_df, columns=df_featureMat_cp.columns)

In [17]:
scaled_df

,sm_efficiency,achieved_occupancy,ipc,issued_ipc,inst_per_warp,branch_efficiency,warp_execution_efficiency,warp_nonpred_execution_efficiency,inst_replay_overhead,issue_slot_utilization,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,0.998561,0.949961,0.387025,0.388792,6.922213e-06,1.000000,0.976005,0.860476,0.000154,0.397164,...,0.250,0.000000,0.000000,0.0,0.0,0.0,0.0,0.245825,0.000000,0.000000
1,0.998709,0.854673,0.333099,0.327202,3.678717e-06,1.000000,0.943727,0.925782,0.000163,0.336364,...,0.125,0.000000,0.000000,0.0,0.0,0.0,0.0,0.205689,0.000000,0.000000
2,0.920774,0.654969,0.144561,0.145777,2.896097e-05,1.000000,0.997587,0.987434,0.009323,0.141760,...,0.250,0.002254,0.000430,0.0,0.0,0.0,0.0,0.062265,0.017720,0.000000
3,0.928899,0.652162,0.155000,0.157595,2.844729e-05,1.000000,0.998655,0.993923,0.009491,0.152949,...,0.250,0.000000,0.000000,0.0,0.0,0.0,0.0,0.066883,0.019463,0.000000
4,0.992334,0.832206,0.220307,0.220992,2.394556e-06,1.000000,0.985961,0.962813,0.001159,0.236521,...,0.250,0.000000,0.000000,0.0,0.0,0.0,0.0,0.130609,0.049184,0.000000
5,0.992418,0.787181,0.396085,0.396824,4.538924e-06,1.000000,0.985393,0.969066,0.000961,0.438230,...,0.250,0.000000,0.000000,0.0,0.0,0.0,0.0,0.315419,0.099679,0.000000
6,0.989778,0.350989,0.229445,0.229587,2.255308e-05,1.000000,1.000000,0.998603,0.001102,0.228452,...,0.375,0.000000,0.000000,0.0,0.0,0.0,0.0,0.136077,0.169411,0.000000
7,0.990176,0.350996,0.227851,0.228240,2.255308e-05,1.000000,1.000000,0.998603,0.000970,0.227112,...,0.375,0.000000,0.000000,0.0,0.0,0.0,0.0,0.134199,0.169327,0.000000
8,0.991136,0.580645,0.286049,0.287490,1.402200e-05,1.000000,1.000000,0.991101,0.001195,0.302778,...,0.250,0.000000,0.000000,0.0,0.0,0.0,0.0,0.267752,0.153800,0.000000
9,0.995167,0.921760,0.078845,0.079641,3.722655e-07,1.000000,1.000000,0.956989,0.002134,0.087032,...,0.250,0.000000,0.000000,0.0,0.0,0.0,0.0,0.033535,0.026139,0.000000


### save the scaler for future use

In [18]:
metrics_scale_dd = {}
index = 0
for col_name in df_featureMat_cp.columns:
    #print col_name
    metrics_scale_dd[col_name] = [scaler.data_min_[index], scaler.data_max_[index]]
    index += 1

#print metrics_scale_dd

with codecs.open('metrics_scaler.json', 'w', encoding='utf-8') as outfile:
    json.dump(metrics_scale_dd, outfile, ensure_ascii=False)

In [19]:
metrics_scale_dd

{'achieved_occupancy': [0.015624000000000001, 0.99568400000000001],
 'atomic_transactions': [0.0, 84557.0],
 'atomic_transactions_per_request': [0.0, 10.347102],
 'branch_efficiency': [0.7297148, 1.0],
 'cf_executed': [1.0, 201988952.0],
 'cf_fu_utilization': [0.10000000000000001, 0.30000000000000004],
 'cf_issued': [1.0, 201988952.0],
 'double_precision_fu_utilization': [0.0, 1.0],
 'dram_read_throughput': [0.0, 83.787367000000003],
 'dram_read_transactions': [0.0, 1111573379.0],
 'dram_utilization': [0.0, 0.90000000000000002],
 'dram_write_throughput': [0.0, 86.053274999999999],
 'dram_write_transactions': [0.0, 9883456.0],
 'ecc_throughput': [0.0, 0.0],
 'ecc_transactions': [0.0, 0.0],
 'eligible_warps_per_cycle': [0.016272999999999999, 10.501155000000001],
 'flop_count_dp': [0.0, 11415296000.0],
 'flop_count_dp_add': [0.0, 1536768000.0],
 'flop_count_dp_fma': [0.0, 4390400000.0],
 'flop_count_dp_mul': [0.0, 1097728000.0],
 'flop_count_sp': [0.0, 51543801856.0],
 'flop_count_sp_add'

In [20]:
metrics_all = []
for metric, _ in metrics_scale_dd.iteritems():
    metrics_all.append(metric)

### save converted metrics for each application

In [21]:
metrics_folder='home-gpu0'
app_metrics_max_dd = {}
for i,f in enumerate(appTrace):
    
    appName = f[8:-4]
    
    csvfile  = metrics_folder + '/' + f
    df_app = read_trace(csvfile)
    
    #
    # find max() for current app metrics
    #
    appMetricMax_dd = {}
    for metric in metrics_all:
        df_metric = df_app.loc[df_app['Metric Name'] == metric]['Avg']
        m_list = [adjust_metric(metric, mVal) for _, mVal in df_metric.iteritems()]
        #print m_list
        appMetricMax_dd[metric] = max(m_list)  # if an applicaiton has multiple kernels, apply max() to find the val
        #break
    
    #print appMetricMax_dd
    #break
    
    app_metrics_max_dd[appName] = appMetricMax_dd # add max metrics for current app to the dd


In [22]:
# application number
appNum = len(app_metrics_max_dd)
print("total apps : {}".format(appNum))

total apps : 76


In [23]:
featMatCols = ['AppName']         
featMatCols.extend(metrics_all)

In [24]:
featMatCols

['AppName',
 'inst_control',
 'stall_constant_memory_dependency',
 'gst_efficiency',
 'inst_fp_64',
 'local_store_transactions',
 'stall_not_selected',
 'global_hit_rate',
 'stall_exec_dependency',
 'shared_store_transactions',
 'branch_efficiency',
 'warp_nonpred_execution_efficiency',
 'inst_bit_convert',
 'gst_throughput',
 'sysmem_read_throughput',
 'issue_slot_utilization',
 'shared_efficiency',
 'shared_load_transactions',
 'flop_count_dp_fma',
 'shared_store_throughput',
 'inst_executed',
 'achieved_occupancy',
 'stall_inst_fetch',
 'dram_read_transactions',
 'dram_write_transactions',
 'stall_sync',
 'gld_transactions_per_request',
 'single_precision_fu_utilization',
 'gst_requested_throughput',
 'sysmem_write_utilization',
 'flop_count_dp_mul',
 'l2_tex_read_throughput',
 'inst_per_warp',
 'inst_integer',
 'inst_issued',
 'ecc_transactions',
 'cf_fu_utilization',
 'l2_tex_read_hit_rate',
 'local_store_transactions_per_request',
 'special_fu_utilization',
 'local_load_transacti

### generate dataframe to store the metrics of each app

In [25]:
df_app = pd.DataFrame(index=np.arange(0, appNum), columns=featMatCols) #  appNum  x featureDim

#
# export data to data frame, so that we can export to csv file easily
#
rowId = 0
for appName, metrics_dd in app_metrics_max_dd.iteritems():    # read all the metrics from 13 test apps
    #print appName
    #print metrics_dd
    
    df_app.loc[rowId, 'AppName'] = appName # fill in kernel name
    # add more metrics according to the column order
    for eachMetric in metrics_all:
        try:
            df_app.loc[rowId, eachMetric] = metrics_dd[eachMetric]
        except Exception as e:
            print e.message, e.args
            print('ERROR!! App = {}'.format(appName))
            sys.exit(0)

    rowId += 1

In [26]:
df_app

,AppName,inst_control,stall_constant_memory_dependency,gst_efficiency,inst_fp_64,local_store_transactions,stall_not_selected,global_hit_rate,stall_exec_dependency,shared_store_transactions,...,tex_fu_utilization,l2_write_throughput,l2_tex_write_throughput,atomic_transactions,gld_efficiency,sm_efficiency,l2_tex_write_hit_rate,shared_store_transactions_per_request,local_memory_overhead,flop_count_sp_fma
0,rodinia_b+tree,5.08e+07,0.00020301,0.225,0,0,0.019835,0.597793,0.0533804,0,...,0.3,0.922203,0.922203,0,0.631774,0.99834,0.99374,0,0,0
1,cudasdk_threadFenceReduction,590848,0.00668755,0.125,0,0,0.00378902,0.5,0.0422594,844,...,0.2,0.0753403,0.0362396,64,1,0.930337,0.969697,0.86653,0,0
2,cudasdk_convolutionFFT2D,4.1984e+06,0.309817,1,0,0,0.0550717,0.75,0.287868,262144,...,0.3,46.8036,46.8036,0,1,0.994889,0.999589,1,0,2.30687e+07
3,shoc_lev1BFS,96399,0.0262821,0.154152,0,0,0.016001,0.718751,0.174219,0,...,0.1,0.159264,0.151634,0,0.133829,0.144637,0.9,0,0,0
4,rodinia_lavaMD,5.74809e+08,1.21e-06,0.25,7.0249e+09,0,8.326e-05,0.0377374,0.939173,2.56262e+06,...,0.1,0.979424,0.979424,0,0.270166,0.997002,1,5.63165,0,0
5,poly_gemm,4.1943e+06,6.778e-05,1,0,0,0.00192948,0.583225,0.0355917,0,...,0.8,135.413,135.413,0,0.825273,0.978355,1,0,0,1.34218e+08
6,rodinia_hybridsort,2.13092e+07,0.00186479,1,0,0,0.0129556,0.5,0.422765,534934,...,0.3,51.897,51.8961,8192,1,0.994671,0.999024,2.69712,0,2.93601e+07
7,cudasdk_MCEstimatePiInlineQ,373152,0.00103251,0.125,0,158640,0.00034957,0.238843,0.0891003,1164,...,0.1,42.6518,42.6518,0,0.125,0.972244,0.298412,1,0.265607,300000
8,cudasdk_MCEstimatePiInlineP,506722,0.0456433,0.232143,0,95480,0.0382432,0.72696,0.115596,1164,...,0.8,4.79259,4.79259,0,0.25,0.941584,0.958763,1,0.0268673,300000
9,cudasdk_shflscan,1.944e+06,0.102906,1,0,0,0.0263921,0.5,0.318093,194880,...,0.3,66.9211,66.9193,0,1,0.981995,1,1.49815,0,0


### apply feature scaling

In [27]:
# the scaling factors are stored in metrics_scale_dd
df_app_scale = df_app.copy()

for metric in metrics_all:
    [x_min, x_max] = metrics_scale_dd[metric]
    
    if x_max == x_min:
        print("[Warning] x_max == x_min ({})".format(metric))
        #x_range = 1e-6    # up-floor, avoid float division by zero
        df_app_scale[metric] = 1.0
    else:
        x_range = x_max - x_min
        df_app_scale[metric] = df_app_scale[metric].apply(lambda x : (x - x_min) / x_range)

[Warning] x_max == x_min (sysmem_read_throughput)
[Warning] x_max == x_min (sysmem_write_utilization)
[Warning] x_max == x_min (ecc_transactions)
[Warning] x_max == x_min (sysmem_read_transactions)
[Warning] x_max == x_min (sysmem_utilization)
[Warning] x_max == x_min (sysmem_read_utilization)
[Warning] x_max == x_min (ecc_throughput)
[Warning] x_max == x_min (sysmem_write_transactions)


In [28]:
# df_app_scale['sysmem_write_utilization']

In [29]:
df_app_scale

,AppName,inst_control,stall_constant_memory_dependency,gst_efficiency,inst_fp_64,local_store_transactions,stall_not_selected,global_hit_rate,stall_exec_dependency,shared_store_transactions,...,tex_fu_utilization,l2_write_throughput,l2_tex_write_throughput,atomic_transactions,gld_efficiency,sm_efficiency,l2_tex_write_hit_rate,shared_store_transactions_per_request,local_memory_overhead,flop_count_sp_fma
0,rodinia_b+tree,2.309659e-02,0.000236,0.225000,0.000000,0.000000,0.050379,0.684131,0.046910,0.000000,...,0.3,0.006123,0.006123,0.000000,0.631774,0.998709,9.937400e-01,0.000000,0.000000e+00,0.000000e+00
1,cudasdk_threadFenceReduction,2.686329e-04,0.007774,0.125000,0.000000,0.000000,0.009624,0.572213,0.034944,0.000101,...,0.2,0.000500,0.000241,0.000757,1.000000,0.928899,9.696970e-01,0.149817,0.000000e+00,0.000000e+00
2,cudasdk_convolutionFFT2D,1.908833e-03,0.360177,1.000000,0.000000,0.000000,0.139878,0.858320,0.299212,0.031246,...,0.3,0.310734,0.310734,0.000000,1.000000,0.995167,9.995885e-01,0.172893,0.000000e+00,2.685552e-03
3,shoc_lev1BFS,4.382805e-05,0.030554,0.154152,0.000000,0.000000,0.040641,0.822558,0.176929,0.000000,...,0.1,0.001057,0.001007,0.000000,0.133829,0.122310,9.000000e-01,0.000000,0.000000e+00,0.000000e+00
4,rodinia_lavaMD,2.613412e-01,0.000001,0.250000,1.000000,0.000000,0.000211,0.043188,1.000000,0.305451,...,0.1,0.006502,0.006502,0.000000,0.270166,0.997335,1.000000e+00,0.973673,0.000000e+00,0.000000e+00
5,poly_gemm,1.906970e-03,0.000079,1.000000,0.000000,0.000000,0.004901,0.667458,0.027770,0.000000,...,0.8,0.899023,0.899023,0.000000,0.825273,0.978193,1.000000e+00,0.000000,0.000000e+00,1.562500e-02
6,rodinia_hybridsort,9.688398e-03,0.002168,1.000000,0.000000,0.000000,0.032906,0.572213,0.444358,0.063761,...,0.3,0.344550,0.344543,0.096881,1.000000,0.994942,9.990244e-01,0.466314,0.000000e+00,3.417968e-03
7,cudasdk_MCEstimatePiInlineQ,1.696558e-04,0.001200,0.125000,0.000000,0.033385,0.000888,0.273338,0.085343,0.000139,...,0.1,0.283169,0.283169,0.000000,0.125000,0.971919,2.984118e-01,0.172893,2.752721e-01,3.492460e-05
8,cudasdk_MCEstimatePiInlineP,2.303844e-04,0.053062,0.232143,0.000000,0.020093,0.097135,0.831953,0.113852,0.000139,...,0.8,0.031818,0.031818,0.000000,0.250000,0.940444,9.587629e-01,0.172893,2.784494e-02,3.492460e-05
9,cudasdk_shflscan,8.838533e-04,0.119633,1.000000,0.000000,0.000000,0.067034,0.572213,0.331733,0.023229,...,0.3,0.444296,0.444284,0.000000,1.000000,0.981929,1.000000e+00,0.259021,0.000000e+00,0.000000e+00


In [30]:
df_app_scale.to_csv('app_proflingmetrics_scaled.csv', encoding='utf-8')